
<h2><center><font color='black'>   Taking a Deep-Learning Dive with Keras </font></center></h2>

open_placeholder_image

In [1]:
from keras import backend as K
import keras.utils.np_utils

from keras.models import Sequential
from keras.layers import Input
from keras.models import Model
from keras.layers.core import Flatten, Dense, Dropout, Activation
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D, ZeroPadding1D
from keras.applications.vgg16 import VGG16
from keras.optimizers import SGD
import numpy as np
import pandas as pd

from IPython.display import SVG
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import cv2
import sys

#import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

Using TensorFlow backend.


In [8]:
import keras.backend as K
K.set_image_data_format('channels_last')

'tf'

In [2]:
from keras.datasets import cifar10
 
# Load pre-shuffled MNIST data into train and test sets
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170500096/170498071 [==============================] - 487s 3us/step


In [9]:
X_train.shape

(50000, 32, 32, 3)

In [12]:
from keras.utils.np_utils import to_categorical
y_test=to_categorical(y_test)
y_train=to_categorical(y_train)

###  Let's work with VGG16 model again

Lets import the model this time



In [16]:
#Get back the convolutional part of a VGG network trained on ImageNet
model_vgg16_conv = VGG16(weights=None, include_top=False)
model_vgg16_conv.summary()

#Create your own input format 
input = Input(shape=(32,32,3),name = 'image_input')  # our input shape: (3,32,32)

#Use the generated model 
output_vgg16_conv = model_vgg16_conv(input)

#Add the fully-connected layers 
x = Flatten(name='flatten')(output_vgg16_conv)
x = Dense(4096, activation='relu', name='fc1')(x)
x = Dense(4096, activation='relu', name='fc2')(x)
x = Dense(10, activation='softmax', name='predictions')(x)

#Create your own model 
my_model = Model(input=input, output=x)  # Using the Functional API

#In the summary, weights and layers from VGG part will be hidden, but they will be fit during the training
my_model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_3 (InputLayer)             (None, None, None, 3) 0                                            
____________________________________________________________________________________________________
block1_conv1 (Convolution2D)     (None, None, None, 64)1792        input_3[0][0]                    
____________________________________________________________________________________________________
block1_conv2 (Convolution2D)     (None, None, None, 64)36928       block1_conv1[0][0]               
____________________________________________________________________________________________________
block1_pool (MaxPooling2D)       (None, None, None, 64)0           block1_conv2[0][0]               
___________________________________________________________________________________________

In [21]:
sgd = SGD(lr=0.1, decay=1e-6, momentum=0.9, nesterov=True)
my_model.compile(optimizer=sgd, loss='categorical_crossentropy',
                 metrics=['accuracy'])


In [22]:
cb_a=keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=2, verbose=0, mode='auto')
# lets look at our results while we are at it
cb_b=keras.callbacks.RemoteMonitor(root='http://localhost:9000')


my_model.fit(X_train, y_train,
          batch_size=128, nb_epoch=10, verbose=1,
          validation_data=(X_test, y_test),callbacks=[cb_a,cb_b])

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 235s - loss: 2.3043 - acc: 0.1000 - val_loss: 2.3043 - val_acc: 0.1000
Epoch 2/10
50000/50000 [==============================] - 193s - loss: 2.3043 - acc: 0.0986 - val_loss: 2.3040 - val_acc: 0.1000
Epoch 3/10
50000/50000 [==============================] - 193s - loss: 2.3044 - acc: 0.0997 - val_loss: 2.3033 - val_acc: 0.1000
Epoch 4/10
50000/50000 [==============================] - 193s - loss: 2.3045 - acc: 0.1000 - val_loss: 2.3035 - val_acc: 0.1000
Epoch 5/10
50000/50000 [==============================] - 193s - loss: 2.3043 - acc: 0.0984 - val_loss: 2.3043 - val_acc: 0.1000
Epoch 6/10
50000/50000 [==============================] - 193s - loss: 2.3044 - acc: 0.0987 - val_loss: 2.3044 - val_acc: 0.1000


**Our Training has flat-lined!  
Let's try again: **


### Code-Along! 

Let's repeat the MNIST training by leveraging [Transfer Learning](http://cs231n.github.io/transfer-learning/)

1) Reinstantiate your VGG16 model: this time set weights='imagenet'    
2) Freeze all previous layers

hint: 
```
for layer in my_model.layers:    
    print(layer.trainable)
        
```
3) We'll want to rescale our image features to be between 0 & 1:   

hint: see Kera's [ImageDataGenerator Function](https://keras.io/preprocessing/image/)

## Setting up a Deep Learning AWS GPU Instance

- Use the **fantastic** EC2 Image that a former student of ours
- It has all sorts of goodies preconfigured for your Deep Learning joy
- You can find his repo [here](https://github.com/Miej/GoDeeper)
- Now follow along with me as we venture Beyond the Wall!
- **Caution**: This will cost $.  If you don't have your AWS credits, be wary.

### Launching the Instance
1. Proceed to the AWS EC2 Console
2. Click "Launch Instance"
3. Click Community AMIs
4. Search for "3e22685e" (make sure you're in the US West (N California) zone or it won't show up)
5. Click Select
6. Select the g2.2xlarge instance and click "Next: Configure Instance Details"
7. Click thru the defaults until Step 5: Tag Instance, give it a name "GPU"
8. Click thru to Step 6: Configure Security Group.  In the rules, select All TCP and Address "My IP", "Review and Launch"!
9. Choose the keypair you should already have and let 'er rip!

### Configuration
1. Log onto your server via:
  - `ssh -i <your_keypair.pem> icarus@<your_public_ip>`
  - Password: `changetheworld`
2. Log out
3. In EC2 Console, reboot your instance (for init scripts to run)
4. Log back in again (#1 above) and let's go!